# **2-3 텐서플로: 분류 응용**

In [5]:
# -*- coding: utf-8 -*-

import pandas as pd
import tensorflow as tf
import random

In [6]:
print(tf.__version__)

2.16.1


In [61]:
infert = pd.read_csv("infert.csv", delimiter = ",")                #  데이터 불러오기

X_data = infert[["parity", "induced", "spontaneous"]] 		 #  X, y 정의
y_data = infert["case"]

In [62]:
train_idx = random.sample(list(range(len(infert))), int(len(infert) * 0.7 ))
train_X = X_data.iloc[train_idx]  			 #  train, test 데이터로 분할
train_y = y_data[train_idx]

test_X = X_data.drop(train_idx)
test_y = y_data.drop(train_idx)

**오류 발생 : module 'tensorflow' has no attribute 'placeholder'**

In [63]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

**해결 : tensorflow version1 사용**

In [77]:
X = tf.placeholder(shape =[None, 3], dtype = tf.float32)
y = tf.placeholder(shape = [None, 1], dtype = tf.float32) #shape 변경

**X와 y의 타입을 맞춰줌으로써 타입 오류 방지**

In [78]:
W1 = tf.Variable(initial_value= tf.random_normal([3, 1]), dtype = tf.float32)
b1 = tf.Variable(initial_value= tf.random_normal([1]), dtype = tf.float32)
L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))

In [80]:
W2 = tf.Variable(initial_value= tf.random_normal([1, 1]), dtype = tf.float32)
b2 = tf.Variable(initial_value= tf.random_normal([1]), dtype = tf.float32)

**이진분류이므로 다중클래스로 분류하는데 사용되는 소프트맥스함수가 아닌 시그모이드함수로 활성함수 변경**

In [81]:
logits = tf.matmul(L1, W2) + b2  # 활성화 함수 적용 전의 값을 logits으로 사용
hypothesis = tf.sigmoid(logits)  # 시그모이드 활성화 함수 적용

In [82]:
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits))
optimizer = tf.train.GradientDescentOptimizer(learning_rate= 0.0001)
train = optimizer.minimize(cost)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [86]:
for i in range(100):   			# 신경망 훈련
  _, cost_val = sess.run([train, cost], feed_dict = {X: train_X, y: train_y.values.reshape(-1, 1)}) # 마찬가지로 y를 2D 배열로 전달
  if i % 20 ==0:
    print("step: ", i, "\t", "cost: ", cost_val)
print("train finished!!")

step:  0 	 cost:  0.92838633
step:  20 	 cost:  0.9266009
step:  40 	 cost:  0.9248271
step:  60 	 cost:  0.923065
step:  80 	 cost:  0.92131394
train finished!!


In [87]:
predict = tf.cast(hypo > 0.5, dtype = tf.float32)   	# 예측
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, y), dtype = tf.float32 ))
print("accuracy: ", sess.run(accuracy, feed_dict={X: test_X, y: test_y.values.reshape(-1, 1)}))

accuracy:  0.6533333
